In [ ]:
from fastai.vision.all import *

### LB Probes

#### Probe 1: study (1,1,2,2 invalid boxes) and image (1,1,2,2 invalid boxes):  

LB:0.0 Expectation:0.0

#### Probe 2: study (1,1,2,2 invalid boxes) and image (none,1,0,0,1,1): 

LB:0.0 Expectation:1/6 (Assuming equal class weights mAP)

LB: 0.0 also supports the existing problem with image level score calculations. Many say it only contributes 0.001 but infact its not contributing at all, not sure when we will have an explanation about this.

#### Probe 3: study (negative,1,1,0,0) and image (1,1,2,2 boxes):  

LB:0.050 Expectation:1/6 (Assuming equal class weights mAP)

This means mAP is weighted by the number of samples of that class in test set. Probe 4 and 5 supports this idea.

#### Probe 4: study (typical,1,1,0,0) and image (1,1,2,2 boxes):  

LB:0.079 Expectation:1/6 (Assuming equal class weights mAP)

This means mAP is weighted.

LB negative probe / LB typical probe = 0.050 / 0.079 = 0.63

no. of negative in train / no. of typical in train =  1676 / 2855 = 0.58

We can see that ratios of LB scores and number of samples are very close. They are not exactly same since class distributions of training and test are not same, we probably have more negatives in test set.

#### Probe 4: study (indeterminate,1,1,0,0) and image (1,1,2,2 boxes):  

LB:0.031 Expectation:1/6 (Assuming equal class weights mAP)

This means mAP is weighted.

LB negative probe / LB indeterminate probe = 0.050 / 0.031 = 1.61

no. of negative in train / no. of indeterminate in train =  1676 / 2855 = 1.59

#### Probe 5: study (atypical,1,1,0,0) and image (1,1,2,2 boxes):  

LB:0.031 Expectation:1/6 (Assuming equal class weights mAP)

This means mAP is weighted.

LB negative probe / LB atypical probe = 0.050 / X = Y (Tomorrow: out of submissions)

no. of negative in train / no. of atypical in train =  1676 / 2855 = 3.53

X should be close to 0.050 / 3.53 ~= 0.14



### What are weights for image and study level predictions?

Above we see that mAP is not weigthed equally, hence not 1/6. We can see the relative contributions within study level classes, but image level classes might be differently weighted compared to study level classes.


We kind of now know that study level predictions are weighted proportionally to their number of samples, if we also assume this for all the 6 classes then here is a rough calculation:


LB (probe 3): 0.050 -  Expectation: no. of negative in train / all train samples = 0.136

LB (probe 4): 0.079 -  Expectation: no. of negative in train / all train samples = 0.230

LB (probe 4): 0.031 -  Expectation: no. of negative in train / all train samples = 0.084


When we assume equal weight between study and image level predictions numbers don't match. Here if we assume a weight of 0.35 for study level predictions:

LB (probe 3): 0.050 -  Expectation: 0.35*(no. of negative in train / all train samples) = 0.047

LB (probe 4): 0.079 -  Expectation: 0.35*(no. of negative in train / all train samples) = 0.080

LB (probe 4): 0.031 -  Expectation: 0.35*(no. of negative in train / all train samples) = 0.029

Again numbers are slightly different because train and test distribution differences.


### Evaluation Formula Hypothesis

Score: 0.35x(Weighted mAP Study) + 0.65x(Weighted mAP Image) ???



P.S. We can't do much probing around images since evaluation is currently broken.

In [ ]:
len(train_image_df)/len(image_)

In [ ]:
train_image_df = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
train_study_df = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")

In [ ]:
train_image_df.head()

In [ ]:
train_study_df.head()

In [ ]:
sample_sub_df = pd.read_csv("../input/siim-covid19-detection/sample_submission.csv")
study_sub_df = sample_sub_df[sample_sub_df['id'].str.contains("study")]
image_sub_df = sample_sub_df[sample_sub_df['id'].str.contains("image")]

In [ ]:
label_counts = Counter(np.where(array(train_study_df.iloc[:,1:]))[1]); label_counts

In [ ]:
(label_counts[2]/(len(train_image_df)+len(train_study_df)))*0.35

In [ ]:
label_counts[0]/label_counts[1], 0.05/0.079

In [ ]:
label_counts[0]/label_counts[3], 0.05/0.031

In [ ]:
def invalid_label_func(o): return " ".join(o.split()[:-4] + ['1','1','2','2'])
def study_label_func(o,l): return f"{l} 1 0 0 1 1"

In [ ]:
study_sub_df.loc[:,'PredictionString'] = study_sub_df['PredictionString'].apply(partial(study_label_func,l='atypical')).values
image_sub_df.loc[:,'PredictionString'] = image_sub_df['PredictionString'].apply(invalid_label_func).values

In [ ]:
sample_sub_df = pd.concat([study_sub_df, image_sub_df])

In [ ]:
sample_sub_df.to_csv("submission.csv",index=False)

In [ ]:
sample_sub_df